In [ ]:
from huggingface_hub import snapshot_download

# one-time download (online machine)
local_dir = "/path/to/gemma2-2b"  # <- choose a real path you own
path = snapshot_download("google/gemma-2-2b", local_dir=local_dir, token="hf_***")  
print("Downloaded to:", path)

/home/joonyoung/.pyenv/versions/edge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 12 files: 100%|██████████| 12/12 [00:00<00:00, 31476.95it/s]

Downloaded to: /home/joonyoung/quantize/gemma2-2b


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # hide GPUs from TensorFlow
# Optional: quieter logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

from ai_edge_torch.generative.examples.gemma import gemma2
from ai_edge_torch.generative.utilities import converter, export_config, loader
from ai_edge_torch.generative.layers import kv_cache as kv_utils

checkpoint_path     = "/path/to/gemma2-2b"  # <— must exist
output_path         = "/tmp"
output_name_prefix  = "gemma2_2b"
prefill_seq_lens    = [512]
kv_cache_max_len    = 1024
quantize            = "dynamic_int8"   # <-- instead of True

model = gemma2.build_2b_model(
    checkpoint_path,
    custom_loader=loader.maybe_get_custom_loader(checkpoint_path, None),
    mask_cache_size=kv_cache_max_len,   # avoids mask_cache assert
)

ec = export_config.ExportConfig(
    mask_as_input=True,
    kvcache_layout=kv_utils.KV_LAYOUT_TRANSPOSED,  # <-- replaces transpose_kv_cache=True
)

converter.convert_to_tflite(
    model,
    output_path=output_path,
    output_name_prefix=output_name_prefix,
    prefill_seq_len=prefill_seq_lens,
    kv_cache_max_len=kv_cache_max_len,
    quantize=quantize,
    lora_ranks=None,
    export_config=ec,
)


I0000 00:00:1757327214.927079    9063 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1757327218.623342    9063 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/joonyoung/.pyenv/versions/edge/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:351: UserWarning: Device capability of jax unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
/home/joonyoung/.pyenv/versions/edge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https:/

INFO:tensorflow:Assets written to: /tmp/tmp0_kwzjuh/assets


INFO:tensorflow:Assets written to: /tmp/tmp0_kwzjuh/assets


: 

In [5]:
print(local_dir)
# Later / offline:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(local_dir, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(local_dir, local_files_only=True)

/home/joonyoung/.cache/huggingface/hub/models--google--gemma-2-2b/snapshots/c5ebcd40d208330abc697524c919956e692655cf


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 24.42it/s]


In [ ]:
"""Example of converting a Gemma2 model to multi-signature tflite model."""

from absl import app
from ai_edge_torch.generative.examples.gemma import gemma2
from ai_edge_torch.generative.utilities import converter
from ai_edge_torch.generative.utilities import export_config
from ai_edge_torch.generative.utilities import loader
import sys

flags = converter.define_conversion_flags(
    "gemma2-2b", default_mask_as_input=True, default_transpose_kv_cache=True
)

DuplicateFlagError: The flag 'checkpoint_path' is defined twice. First from ai_edge_torch.generative.utilities.converter, Second from ai_edge_torch.generative.utilities.converter.  Description from first occurrence: The path to the model checkpoint, or directory holding the checkpoint.

In [7]:
def main(_):
  checkpoint_path = flags.FLAGS.checkpoint_path
  pytorch_model = gemma2.build_2b_model(
      checkpoint_path,
      custom_loader=loader.maybe_get_custom_loader(
          checkpoint_path, flags.FLAGS.custom_checkpoint_loader
      ),
      mask_cache_size=converter.get_mask_cache_size_from_flags(),
  )
  converter.convert_to_tflite(
      pytorch_model,
      output_path=flags.FLAGS.output_path,
      output_name_prefix=flags.FLAGS.output_name_prefix,
      prefill_seq_len=flags.FLAGS.prefill_seq_lens,
      kv_cache_max_len=flags.FLAGS.kv_cache_max_len,
      quantize=flags.FLAGS.quantize,
      lora_ranks=flags.FLAGS.lora_ranks,
      export_config=export_config.get_from_flags(),
  )


if __name__ == '__main__':
  # Keep only argv[0] so Abseil sees no unknown flags
  sys.argv = sys.argv[:1]
  app.run(main)


FileNotFoundError: File or directory not found: /home/joonyoung/Downloads/llm_data/gemma2-2b